In [3]:
import sys
import os
import pandas as pd

import numpy as np
import seaborn as sns

import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
workfolder = os.getenv('Projects') + '/Uber_market'
os.chdir(workfolder)

In [5]:
#Import Census Tract - Community District correspondence
ct_nta = pd.read_csv('Data/Non538_Input/ct_cd_corresp2.csv').rename(columns={'tract_fips_10': 'GEOID'})
ct_nta.GEOID = ct_nta.GEOID.astype(str).str[:-2]

## I - Education
### 1 - Import & slice

In [6]:
#Import ACS data on education
acs2 = pd.read_csv('Data/Non538_Input/Census/ACS_13_5YR_B15003_with_ann.csv')
acs2 = acs2.loc[acs2['GEO.id'] != 'Id']

#Select columns
cols = [c for c in acs2.columns if c[:3] != 'GEO']
acs2[cols] = acs2[cols].apply(pd.to_numeric)

In [7]:
#Rename columns
for i in range(0,14):
    if i < 8:
        acs2['educ_{}'.format(i)] = acs2['HD01_VD0{}'.format(i+2)]
    else:
        acs2['educ_{}'.format(i)] = acs2['HD01_VD{}'.format(i+2)]

acs2['educ_13.5'] = acs2['HD01_VD16']
acs2['educ_14'] = acs2['HD01_VD17'] + acs2['HD01_VD18']
acs2['educ_14.5'] = acs2['HD01_VD19']

for i in [15,16]:
    acs2['educ_{}'.format(i)] = acs2['HD01_VD{}'.format(i+5)]

acs2['educ_18'] = acs2['HD01_VD22']
acs2['educ_20'] = acs2['HD01_VD23'] + acs2['HD01_VD24']
acs2['educ_23'] = acs2['HD01_VD25']

acs2['tot_educ'] = acs2['HD01_VD01']

In [8]:
#Discrepancy between sum of columns and total number
(acs2[[c for c in acs2.columns if c[:3] == 'edu']].sum(axis=1) - acs2['tot_educ'] == 0).value_counts()

True    2167
dtype: int64

### 2 - Compute shares

In [9]:
#Rename columns
acs2b = acs2[['GEO.id2', 'GEO.display-label', 'tot_educ'] + 
             [c for c in acs2.columns if c[:4] == 'educ']].rename(columns={'GEO.id2': 'GEOID'})

#Share of each education category
for var in [c for c in acs2b.columns if c[:4] == 'educ']:
    acs2b['r{}'.format(var)] = acs2b['{}'.format(var)]/acs2b['tot_educ']

#Add geolocal information
acs2b = acs2b.merge(ct_nta[['NTACode', 'GEOID']], on='GEOID', how='left')

In [10]:
acs2b.to_csv('Data/Output/acs_educ.csv')

## II - Income
### 1 - Import & slice

In [11]:
#Import Income data
acs5 = pd.read_csv('Data/Non538_Input/Census/ACS_13_5YR_B19001_with_ann.csv')

In [12]:
#Select columns
brackets = ['hh_tot', 'inc10', 'inc15', 'inc20', 'inc25', 'inc30', 'inc35', 'inc40', 'inc45',
            'inc50', 'inc60', 'inc75', 'inc100', 'inc125', 'inc150', 'inc200', 'inc200+']
cols = [c for c in acs5.columns if c[2:4] == '01']

#Convert to numeric
acs5[brackets] = acs5[cols].convert_objects(convert_numeric=True)
acs5 = acs5.loc[(acs5['GEO.id'] != 'Id') & (acs5.hh_tot != 0)]

#Rename columns
acs5b = acs5[['GEO.id2', 'GEO.display-label'] + brackets].rename(columns={'GEO.id2': 'GEOID'})

/Users/arnoamabile/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


### 2 - Compute shares

In [13]:
#Compute share of each bracket population
for var in [c for c in acs5b.columns if c[:3] == 'inc']:
    acs5b.loc[:,'r{}'.format(var)] = acs5b.loc[:,'{}'.format(var)]/acs5b.loc[:,'hh_tot']

#Add geolocal information
acs5b = acs5b.merge(ct_nta[['NTACode', 'GEOID']], on='GEOID', how='left')

In [14]:
acs5b.to_csv('Data/Output/acs_inc.csv')

## III - Age
### 1 - Import & slice

In [16]:
#Import Age Census data
acs11 = pd.read_csv('Data/Non538_Input/Census/ACS_13_5YR_S0101_with_ann.csv')

In [17]:
#Rename columns
cols = [c for c in acs11.columns if c != 'GEO.id2']
acs11[cols] = acs11[cols].convert_objects(convert_numeric=True)
acs11 = acs11.rename(columns = {'HC01_EST_VC01': 'tot_pop'})
acs11 = acs11.loc[(acs11['GEO.id'] != 'Id') & (acs11.tot_pop != 0)]

#Rename columns
acs11['male'], acs11['female'] = acs11['HC02_EST_VC01'], acs11['HC03_EST_VC01']
acs11['rmale'], acs11['rfemale'] = acs11['male']/acs11.tot_pop, acs11['female']/acs11.tot_pop

/Users/arnoamabile/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


### 2 - Compute shares

In [18]:
#Compute share of each bracket
for i in range(3,21):
    if i < 10:
        acs11['rage_ud{}'.format(4 + 5*(i-3))] = acs11['HC01_EST_VC0{}'.format(i)]/100
    else:
        acs11['rage_ud{}'.format(4 + 5*(i-3))] = acs11['HC01_EST_VC{}'.format(i)]/100
    acs11['age_ud{}'.format(4 + 5*(i-3))] = acs11['rage_ud{}'.format(4 + 5*(i-3))]*acs11['tot_pop']

acs11 = acs11.rename(columns={'age_ud89': 'age_ov85','rage_ud89':'rage_ov85'})

#Rename columns
acs11 = acs11[['GEO.id2', 'tot_pop', 'male', 'female'] + [c for c in acs11.columns if c[:3] == 'age'] + 
              [c for c in acs11.columns if c[0] == 'r']].rename(columns={'GEO.id2': 'GEOID'})

In [19]:
#Add geolocal information
acs11b = acs11.merge(ct_nta[['NTACode', 'GEOID']], on='GEOID', how='left')

In [21]:
acs11b.to_csv('Data/Output/acs_age.csv')